In [25]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('CaseStudy.db')
cursor = conn.cursor()

#cursor.execute(''' DROP TABLE CUSTOMER''')

cursor.execute('''CREATE TABLE CUSTOMER (customer_email CHAR(255), net_revenue FLOAT, YEAR INTEGER)''')

conn.commit()

In [26]:
# read csv to dataframe and then to CUSTOMER table in database
read_data = pd.read_csv('casestudy.csv', usecols=['customer_email','net_revenue','year'])
read_data.to_sql('CUSTOMER', conn, if_exists='append', index = False)
conn.commit()

In [27]:
# total revenue for current year (2017)
cursor.execute('''
SELECT round(sum(net_revenue), 2)
FROM CUSTOMER
WHERE year = 2017
''')
print("Total Revenue for 2017: $",cursor.fetchall())

Total Revenue for 2017: $ [(31417495.03,)]


In [28]:
# Total new Customer Revenue e.g. new customer revenue from 2017 only
cursor.execute('''
SELECT round(sum(net_revenue),2)
FROM CUSTOMER 
WHERE year = 2017 AND customer_email NOT IN(
    SELECT DISTINCT customer_email
    FROM CUSTOMER
    WHERE year < 2017
)
''')
print("New Customer Revenue 2017: $", cursor.fetchall())

New Customer Revenue 2017: $ [(28676607.64,)]


In [31]:
# Revenue of existing customers for current year i.e.customers exisiting prior to 2017 and revenue from 2017
cursor.execute('''
SELECT round(sum(net_revenue),2)
FROM CUSTOMER
WHERE year = 2017 AND customer_email IN(
    SELECT DISTINCT customer_email
    FROM CUSTOMER
    WHERE year < 2017
)
''')
print("Existing Customer Revenue 2017: $", cursor.fetchall())

Existing Customer Revenue 2017: $ [(2740887.39,)]


In [32]:
# Revenue lost from attrition e.g. revenue lost between 2016 and 2017
cursor.execute('''
SELECT (SELECT round(sum(net_revenue),2)
FROM CUSTOMER 
WHERE year = 2017) - (SELECT round(sum(net_revenue),2)
FROM CUSTOMER 
WHERE year = 2016) 
''')
print("Total Revenue lost in attrition between 2016 and 2017): $", cursor.fetchall())

Total Revenue lost in attrition between 2016 and 2017): $ [(5686551.440000001,)]


In [33]:
# Existing Customer Revenue Current Year i.e.customer email from 2017 and revenue from 2017
cursor.execute('''
SELECT round(sum(net_revenue),2)
FROM CUSTOMER
WHERE year = 2017 AND customer_email IN(
    SELECT DISTINCT customer_email
    FROM CUSTOMER as C
    WHERE year = 2017
)
''')
print("Revenue for 2017 from exsiting customers:", cursor.fetchall())

Revenue for 2017 from exsiting customers: [(31417495.03,)]


In [34]:
# Existing Customer Revenue Prior Year i.e. current customer email from 2017 but revenue for 2016
cursor.execute('''
SELECT round(sum(net_revenue),2)
FROM CUSTOMER
WHERE year = 2016 AND customer_email IN(
    SELECT DISTINCT customer_email
    FROM CUSTOMER
    WHERE year = 2017
)
''')
print("Revenue for 2016 from exsiting customers:", cursor.fetchall())

Revenue for 2016 from exsiting customers: [(2620648.65,)]


In [35]:
# total number of customers in current year i.e. 2017
cursor.execute('''
SELECT count(DISTINCT customer_email)
FROM CUSTOMER
WHERE year = 2017
''')
print("Total Customers in 2017: ", cursor.fetchall())

Total Customers in 2017:  [(249987,)]


In [36]:
# total number of customers in previous year i.e. 2016
cursor.execute('''
SELECT count(DISTINCT customer_email)
FROM CUSTOMER
WHERE year = 2016
''')
print("Total Customers in 2016: ", cursor.fetchall())

Total Customers in 2016:  [(204646,)]


In [37]:
# new total customers in 2017 only
cursor.execute('''
SELECT count(DISTINCT customer_email)
FROM CUSTOMER 
WHERE year = 2017 and customer_email NOT IN(
    SELECT DISTINCT customer_email
    FROM CUSTOMER
    WHERE year < 2017
)
''')
print("New Customers in 2017 only: ", cursor.fetchall())

New Customers in 2017 only:  [(228262,)]


In [38]:
# lost customers i.e. customers in previous years but not in 2017
cursor.execute('''
SELECT count(DISTINCT customer_email)
FROM CUSTOMER
WHERE year < 2017 AND customer_email NOT IN (
    SELECT DISTINCT customer_email
    FROM CUSTOMER
    WHERE year = 2017
)
''')
print("Customers lost in 2017: ", cursor.fetchall())
conn.close()

Customers lost in 2017:  [(354631,)]
